# **Processing and Transforming PSA, Person, and Institution Data in PySpark**

This document outlines the step-by-step process for loading, filtering, joining, deduplicating, and pivoting data from PSA, Person, and Institution datasets using PySpark.

## 1. Load Data from Parquet Files
We begin by reading three datasets from Parquet files into PySpark DataFrames.

In [0]:
psa_df = spark.read.parquet("/mnt/ci-mvi/Processed/SVeteran.SMVIPersonSiteAssociation/")
person_df = spark.read.parquet("/mnt/ci-mvi/Processed/SVeteran.SMVIPerson")
institution_df = spark.read.parquet("/mnt/ci-mvi/Raw/NDim.MVIInstitution/")


In [0]:
psa_df.filter((psa_df.MVIPersonICN == 1035192139) & (psa_df.TreatingFacilityPersonIdentifier == 10050329)).display()

- psa_df: Contains information related to Treating Facility and Person Identifiers.

- person_df: Contains Person ICN and their status.

- institution_df: Contains mappings of Institution SIDs to Institution Codes.

## 2. Define Window Specifications

Window functions are used to rank records within a partitioned group.

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, rank, first, row_number

psa_window_spec = Window.partitionBy("MVIPersonICN", "MVITreatingFacilityInstitutionSID").orderBy("CorrelationModifiedDateTime")
person_window_spec = Window.partitionBy("MVIPersonICN").orderBy("calc_IngestionTimestamp")


- psa_window_spec: Partitions data by MVIPersonICN and MVITreatingFacilityInstitutionSID, ordering by CorrelationModifiedDateTime to select the latest record per partition.

- person_window_spec: Partitions data by MVIPersonICN, ordering by calc_IngestionTimestamp to pick the most recent ingestion record.

## 3. Process PSA Data

This step filters and ranks PSA records to retain only the most recent entry per MVIPersonICN and MVITreatingFacilityInstitutionSID.

In [0]:
psa_df = (
    psa_df.filter(
        (col("MVITreatingFacilityInstitutionSID").isin([5667, 6061, 6722])) &
        ((col("ActiveMergedIdentifier").isNull()) | (col("ActiveMergedIdentifier") == 'Active')) &
        col("MVIPersonICN").isNotNull()
    )
    # .select("MVIPersonICN", "TreatingFacilityPersonIdentifier", "MVITreatingFacilityInstitutionSID", "CorrelationModifiedDateTime")
    # .distinct()
    .withColumn("rank", rank().over(psa_window_spec))
    .filter(col("rank") == 1)
    .select("MVIPersonICN", "TreatingFacilityPersonIdentifier", "MVITreatingFacilityInstitutionSID", "CorrelationModifiedDateTime")
)


###  Filtering Criteria:
  -   Keeps only Institution SIDs 5667, 6061, and 6722.

  -   Excludes records where ActiveMergedIdentifier is not 'Active' or NULL.

  - Ensures MVIPersonICN is not NULL.

###  Ranking:

  -   Assigns a rank based on CorrelationModifiedDateTime (latest first) within each partition.

### Selection:

  - Retains only the highest-ranked record for each partition.

## 4. Process Person Data

Similar to PSA processing, this step filters and retains only the most recent Person ICN record.

In [0]:
person_df = (
    person_df.withColumn("rank", rank().over(person_window_spec))
    .filter(col("rank") == 1)
    .select("MVIPersonICN", "ICNStatus")
)


### Ranking:

  - Assigns a rank based on calc_IngestionTimestamp (latest first) for each MVIPersonICN.

### Selection:

  -  Keeps only the latest record per MVIPersonICN.



## 5. Join PSA Data with Institution Data

This step maps the MVITreatingFacilityInstitutionSID in psa_df to InstitutionCode from institution_df

In [0]:
relationship_df = (
    psa_df.join(institution_df, psa_df["MVITreatingFacilityInstitutionSID"] == institution_df["MVIInstitutionSID"], "left")
    .select(psa_df["MVIPersonICN"], institution_df["InstitutionCode"], psa_df["TreatingFacilityPersonIdentifier"], psa_df["CorrelationModifiedDateTime"].alias("Last_Modified"))
).cache()


In [0]:
relationship_df.limit(5).display()

### Performs a left join between psa_df and institution_df.

- **Retains:**

    - MVIPersonICN

    - InstitutionCode

    - TreatingFacilityPersonIdentifier

    - CorrelationModifiedDateTime

### 6. Identify Duplicate Records

Duplicates are identified based on two different criteria.

In [0]:
duplicate_iens = relationship_df.groupBy("MVIPersonICN", "InstitutionCode").count().filter(col("count") > 1).withColumnRenamed("count", "count_1")
duplicate_icns = relationship_df.groupBy("InstitutionCode", "TreatingFacilityPersonIdentifier").count().filter(col("count") > 1).withColumnRenamed("count", "count_2")


- duplicate_iens: Finds duplicate MVIPersonICN within the same InstitutionCode.

- duplicate_icns: Identifies duplicate TreatingFacilityPersonIdentifier within an InstitutionCode

### 7. Remove Duplicate Records

This step removes records that were marked as duplicates.

In [0]:
correlations = (
    relationship_df
    .join(duplicate_iens, ["MVIPersonICN", "InstitutionCode"], "left")
    .join(duplicate_icns, ["InstitutionCode", "TreatingFacilityPersonIdentifier"], "left")
).cache()

unduped_correlations = (
    correlations
    .filter(col("count_1").isNull())
    .filter(col("count_2").isNull())
).cache()


- Left joins with duplicate_iens and duplicate_icns to mark duplicates.

- Filters out any records where a duplicate count exists (count is not NULL).

In [0]:
unduped_correlations.limit(5).display()

### 8. Pivot the Data for Final Transformation
The data is reshaped into a wide format using pivoting.

In [0]:
icn_master = (
    unduped_correlations.join(person_df, ["MVIPersonICN"], "left")
    .groupBy("MVIPersonICN")
    .pivot("InstitutionCode", ["200CORP", "200DOD", "200VETS"])
    .agg(first("TreatingFacilityPersonIdentifier"))
    .join(person_df.select("MVIPersonICN", "ICNStatus"), ["MVIPersonICN"], "left")
    .withColumnRenamed("ICNStatus", "icn_status")
    .withColumnRenamed("200CORP", "participant_id")
    .withColumnRenamed("200DOD", "edipi")
    .withColumnRenamed("200VETS", "va_profile_id")
).cache()


In [0]:
icn_master.display()

Joins unduped_correlations with person_df to bring in ICNStatus.

Pivots on InstitutionCode to create separate columns for:

- 200CORP → 6061 → participant_id

- 200DOD → 5667 → edipi 

- 200VETS → 6722 → va_profile_id

- Renames columns for clarity.



Final Output
The final dataset icn_master contains:

MVIPersonICN: Unique person identifier.

participant_id: Corresponding Treating Facility ID for 200CORP.

edipi: Corresponding Treating Facility ID for 200DOD.

va_profile_id: Corresponding Treating Facility ID for 200VETS.

icn_status: ICN status for the person.

Summary
This PySpark workflow processes three datasets (PSA, Person, Institution) to:

Load data from Parquet files.

Filter and retain the latest records using window functions.

Join PSA data with institution mappings.

Identify and remove duplicate records.

Pivot data to create a structured master dataset.

This workflow ensures efficient processing, eliminates duplicates, and provides a final structured dataset ready for further analysis.

### Duplicate Identities

In [0]:
duplicate_correlations = (
    correlations
    .filter(col("count_1").isNotNull() | col("count_2").isNotNull())
).cache()

In [0]:
duplicate_correlations.display()

In [0]:
duplicate_correlations.display()


In [0]:
duplicate_correlations.filter(duplicate_correlations.MVIPersonICN == 1035192139).display()

In [0]:
duplicate_correlations.filter(duplicate_correlations.MVIPersonICN == 1035192139).display()


In [0]:
unduped_correlations.filter(duplicate_correlations.TreatingFacilityPersonIdentifier == 10050359).display()


In [0]:
duplicate_correlations.filter(duplicate_correlations.TreatingFacilityPersonIdentifier == 10050359).display()


In [0]:
psa_df.filter(psa_df.TreatingFacilityPersonIdentifier == 10050359).display()

In [0]:
%sql
SELECT *
FROM delta.`/mnt/Patronage/identity_correlations` --`/mnt/Patronage/distinct_identity_correlations`
WHERE MVIPersonICN IN (1062563044, 1005082557, 1035192139, 1008247960,1014287787,1079672752)

In [0]:
relationship_df.createOrReplaceTempView("relationship_df")
unduped_correlations.createOrReplaceTempView("unduped_correlations")
person.createOrReplaceTempView('person')

In [0]:
%sql
SELECT count(*), 'dist_count'
FROM delta.`/mnt/Patronage/distinct_identity_correlations`
UNION
SELECT count(*), 'old_count'
FROM delta.`/mnt/Patronage/identity_correlations`
-- 61276959

In [0]:
%sql
SELECT *
FROM delta.`/mnt/Patronage/distinct_identity_correlations` 
WHERE MVIPersonICN NOT IN (SELECT MVIPersonICN
FROM delta.`/mnt/Patronage/identity_correlations`)

In [0]:
relationship_df.createOrReplaceTempView("relationship_df")
unduped_correlations.createOrReplaceTempView("unduped_correlations")
person_df.createOrReplaceTempView('person')

In [0]:
icn_master = spark.sql("""
    with 200CORP as (
    select
        MVIPersonICN,
        TreatingFacilityPersonIdentifier, 
        Last_Modified
    from
        unduped_correlations
    where
        InstitutionCode = '200CORP'
    ),
    200DOD as (
    select
        MVIPersonICN,
        TreatingFacilityPersonIdentifier,
        Last_Modified
    from
        unduped_correlations
    where
        InstitutionCode = '200DOD'
    ),
    200VETS as (
    select
        MVIPersonICN,
        TreatingFacilityPersonIdentifier,
        Last_Modified
    from
        unduped_correlations
    where
        InstitutionCode = '200VETS'
    ),
    unique_ICNs as (
    select MVIPersonICN, max(Last_Modified) as record_updated_date from relationship_df group by 1
    ),
    person as (
        select MVIPersonICN, ICNStatus from person
    )
    select
    a.MVIPersonICN,
    b.TreatingFacilityPersonIdentifier as participant_id,
    c.TreatingFacilityPersonIdentifier as edipi,
    d.TreatingFacilityPersonIdentifier as va_profile_id,
    e.ICNSTatus as icn_status,
    a.record_updated_date
    from
    unique_ICNs a
    left join 200CORP b on a.MVIPersonICN = b.MVIPersonICN
    left join 200DOD c on a.MVIPersonICN = c.MVIPersonICN
    left join 200VETS d on a.MVIPersonICN = d.MVIPersonICN
    left join person e on a.MVIPersonICN = e.MVIPersonICN
    """)

In [0]:
dbutils.fs.rm('/mnt/Patronage/distinct_identity_correlations', True)

In [0]:
%sql
drop table delta.`/mnt/Patronage/distinct_identity_correlations`

In [0]:
icn_master.write.format("delta").mode("overwrite").save(
        "/mnt/Patronage/distinct_identity_correlations"
    )

In [0]:
%sql
SELECT *
FROM delta.`/mnt/Patronage/distinct_identity_correlations`
WHERE MVIPersonICN == 1006433939

In [0]:
%sql
SELECT *
FROM delta.`/mnt/Patronage/identity_correlations`
WHERE MVIPersonICN ==  1008247960

In [0]:
icn_master.createOrReplaceTempView("icn_mast")

In [0]:
%sql
SELECT *
FROM icn_mast
WHERE MVIPersonICN == 1006433939

In [0]:
%sql
SELECT * FROM (DESCRIBE HISTORY delta.`/mnt/Patronage/identity_correlations`) 

In [0]:
%sql
select current_timestamp()